In [1]:
import os, sys, MySQLdb, warnings
import numpy as np
import pandas as pd
import shutil
from datetime import datetime as datetime

In [2]:
#文件目录
File_download_path='I://百度云下载//'
MySQL_DataPath='I://MySQL//Data//aiot//'
DateRange=['2018-8-4:2018-8-5','2018-8-7:2018-8-8','2018-8-16:2018-8-17']
Code_list=['ZD-124-01','ZD-117-01','ZD-100-01','ZD-83-01','ZD-67-01']

In [3]:
#启动SQL服务
def Start_MySQL():
    conn = MySQLdb.connect(
        host = '127.0.0.1',
        port = 3306,
        user = 'root',
        passwd='1234',
        db = 'aiot',
        charset='utf8'
        )
    cur = conn.cursor()
    return(conn,cur)
#退出SQL服务
def Stop_MySQL(conn,cur):
    cur.close()
    conn.close()

In [4]:
def get_date_list(DateRange):
    Decode_list=pd.DataFrame([x.split(':') for x in DateRange],columns=('Start_Time','End_Time'))
    AllPeriod_list=list()
    for index, row in Decode_list.iterrows():
        OnePeriod_list=[x.strftime('%Y_%m_%d') for x in pd.date_range(row['Start_Time'],row['End_Time'])]
        AllPeriod_list=AllPeriod_list+OnePeriod_list
    Date_list=pd.DataFrame(AllPeriod_list,columns=['Date'])
    Date_list=Date_list.drop_duplicates()
    return(Date_list)

def convert_date(s_date): # 数据库中的日期是8_13这种而不是08_13，故需做此转换
    cells = s_date.split('_')
    i_year = cells[0]
    i_month = int(cells[1])
    i_day = int(cells[2])
    s_date = i_year+'_'+str(i_month)+'_'+str(i_day)
    return(s_date)


def get_file_list(Date_list,Code_list,cur):
    Serial_list='exp_'+pd.DataFrame([SQLget_Instru_serial(cur, x) for x in Code_list],columns=['Serial'])+'_'
    Serial_list=Serial_list.drop_duplicates()
    FileName_list=[]
    for x,y in Serial_list.iterrows():
       for i,j in Date_list.iterrows():
           FileName_list=FileName_list+[y['Serial']+convert_date(j['Date'])]
    FileName_list=pd.DataFrame(FileName_list,columns=['FileName'])
    return(FileName_list)

def File_Path(File_download_path,MySQL_DataPath,FileName_list):
    ibd_Old_Path=File_download_path+FileName_list+'.ibd'
    ibd_New_Path=MySQL_DataPath+FileName_list+'.ibd'
    frm_Old_Path=File_download_path+FileName_list+'.frm'
    frm_New_Path=MySQL_DataPath+FileName_list+'.frm'
    ibdFile_Path=pd.merge(ibd_Old_Path,ibd_New_Path,left_index=True,right_index=True)
    frmFile_Path=pd.merge(frm_Old_Path,frm_New_Path,left_index=True,right_index=True)
    File_Path=pd.merge(ibdFile_Path,frmFile_Path,left_index=True,right_index=True)
    File_Path.columns=['ibd_Old_Path','ibd_New_Path','frm_Old_Path','frm_New_Path']
    return(File_Path)

In [5]:
#创建空白表格
def Creat_BlankTable(MySQL_DataPath,FileName_list,cur):
    for index, row in FileName_list.iterrows():
        TargetFilePath=MySQL_DataPath+row['FileName']+'.ibd'
        if  os.path.exists(TargetFilePath):
            print('文件',row['FileName'],'已存在')
        else:
            SQLCreat_BlankTable(row['FileName'],cur)
            SQLDiscard_ibd_frm(row['FileName'],cur)

#连接ibd与frm
def Connect_ibd_frm(FileName_list,cur):
    for index, row in FileName_list.iterrows():
        SQLCombine_ibd_frm(row['FileName'],cur)


In [6]:
#从楼层传感器编号转换为数据序列编号
def SQLget_Instru_serial(cursor, code):
    sql = '''select Instru_serial from exp_instrument_info where number = '%s' ''' % ( code)
    try:
        cursor.execute(sql)
        results = cursor.fetchone()
        return results
    except Exception as ex:
        print(Exception, ':', ex)

#创建空白表格
def SQLCreat_BlankTable(s_code,cur):
    sql='''CREATE TABLE `aiot`.`%s` (
         `cur_time` char(15) NOT NULL COMMENT 'timeInfo XX:XX:XX.XXXXX',
         `data1` double DEFAULT NULL,
         `data2` double DEFAULT NULL,
         `data3` double DEFAULT NULL,
         `data4` double DEFAULT NULL,
         `data5` double DEFAULT NULL,
         `data6` double DEFAULT NULL,
         `data7` double DEFAULT NULL,
         `data8` double DEFAULT NULL,
         PRIMARY KEY (`cur_time`)
    ) ENGINE=InnoDB DEFAULT CHARSET=latin1 ROW_FORMAT=COMPACT;'''%(s_code)
    try:
        cur.execute(sql)
        print(s_code , 'is created')
    except Exception as ex:
        print(Exception, ':', ex)

#将移动后的ibd文件与frm文件合并
def SQLCombine_ibd_frm(s_code,cur):
    sql='''alter table %s import tablespace;'''%(s_code)
    try:
        cur.execute(sql)
        print(s_code , 'is connetd')
    except Exception as ex:
        print(Exception, ':', ex)

#将ibd与frm文件解耦
def SQLDiscard_ibd_frm(s_code,cur):
    sql='''alter table %s discard tablespace;''' %(s_code)
    try:
        cur.execute(sql)
        print(s_code , 'is discarded')
    except Exception as ex:
        print(Exception, ':', ex)

In [7]:
#从下载路径复制文件到MySQL数据库
def Move_Data(File_Path):
    for x,y in File_Path.iterrows():
        if os.path.exists(y['ibd_Old_Path']):
            shutil.move(y['ibd_Old_Path'],y['ibd_New_Path'])#移动ibd文件
            shutil.move(y['frm_Old_Path'],y['frm_New_Path'])#移动frm文件
        else :
            print('未下载',y['ibd_Old_Path'])

In [8]:
#启动MySQL
conn, cur = Start_MySQL()
#生成文件路径
Date_list=get_date_list(DateRange)
FileName_list=get_file_list(Date_list,Code_list,cur)
File_Path=File_Path(File_download_path,MySQL_DataPath,FileName_list)
#创建空白表格
Creat_BlankTable(MySQL_DataPath,FileName_list,cur)
#移动文件
Move_Data(File_Path)
#关联文件
Connect_ibd_frm(FileName_list,cur) 
#关闭服务
Stop_MySQL(conn,cur)

文件 exp_10496_2018_8_4 已存在
文件 exp_10496_2018_8_5 已存在
exp_10496_2018_8_6 is created
exp_10496_2018_8_6 is discarded
文件 exp_10496_2018_8_7 已存在
文件 exp_10496_2018_8_8 已存在
文件 exp_10496_2018_8_16 已存在
文件 exp_10496_2018_8_17 已存在
文件 exp_10108_2018_8_4 已存在
文件 exp_10108_2018_8_5 已存在
exp_10108_2018_8_6 is created
exp_10108_2018_8_6 is discarded
文件 exp_10108_2018_8_7 已存在
文件 exp_10108_2018_8_8 已存在
文件 exp_10108_2018_8_16 已存在
文件 exp_10108_2018_8_17 已存在
文件 exp_10087_2018_8_4 已存在
文件 exp_10087_2018_8_5 已存在
exp_10087_2018_8_6 is created
exp_10087_2018_8_6 is discarded
文件 exp_10087_2018_8_7 已存在
文件 exp_10087_2018_8_8 已存在
文件 exp_10087_2018_8_16 已存在
文件 exp_10087_2018_8_17 已存在
文件 exp_10129_2018_8_4 已存在
文件 exp_10129_2018_8_5 已存在
exp_10129_2018_8_6 is created
exp_10129_2018_8_6 is discarded
文件 exp_10129_2018_8_7 已存在
文件 exp_10129_2018_8_8 已存在
文件 exp_10129_2018_8_16 已存在
文件 exp_10129_2018_8_17 已存在
文件 exp_10131_2018_8_4 已存在
文件 exp_10131_2018_8_5 已存在
exp_10131_2018_8_6 is created
exp_10131_2018_8_6 is discarded
文件 exp